# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one of the
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using DFTK
using LinearAlgebra
using ASEconvert

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    a = 7.65339
    lattice = a * Matrix(I, 3, 3)
    Al = ElementPsp(:Al; psp=load_psp("hgh/lda/al-q3"))
    atoms     = [Al, Al, Al, Al]
    positions = [[0.0, 0.0, 0.0], [0.0, 0.5, 0.5], [0.5, 0.0, 0.5], [0.5, 0.5, 0.0]]
    unit_cell = periodic_system(lattice, atoms, positions)

    # Make supercell in ASE:
    # We convert our lattice to the conventions used in ASE, make the supercell
    # and then convert back ...
    supercell_ase = convert_ase(unit_cell) * pytuple((repeat, 1, 1))
    supercell     = pyconvert(AbstractSystem, supercell_ase)

    # Unfortunately right now the conversion to ASE drops the pseudopotential information,
    # so we need to reattach it:
    supercell = attach_psp(supercell; Al="hgh/lda/al-q3")

    # Construct an LDA model and discretise
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    model = model_LDA(supercell; temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end;

    CondaPkg Found dependencies: /home/runner/.julia/packages/ASEconvert/CNQ1A/CondaPkg.toml
    CondaPkg Found dependencies: /home/runner/.julia/packages/PythonCall/wXfah/CondaPkg.toml
    CondaPkg Dependencies already up to date


As part of the code we are using a routine inside the ASE,
the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
for creating the supercell and make use of the two-way interoperability of
DFTK and ASE. For more details on this aspect see the documentation
on Input and output formats.

Write an example supercell structure to a file to plot it:

In [2]:
setup = aluminium_setup(5)
convert_ase(periodic_system(setup.model)).write("al_supercell.png")

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123


Python: None

<img src="https://docs.dftk.org/stable/examples/al_supercell.png" width=500 height=500 />

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trivial statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [3]:
self_consistent_field(aluminium_setup(1); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.298569952433                   -0.85    5.2         
  2   -8.300222220683       -2.78       -1.26    1.1   86.7ms
  3   -8.300438496722       -3.66       -1.89    2.1   99.0ms
  4   -8.300461639329       -4.64       -2.75    3.0    112ms
  5   -8.300464262788       -5.58       -3.15    2.9    210ms
  6   -8.300464436023       -6.76       -3.32    5.1    119ms
  7   -8.300464535031       -7.00       -3.46    9.1    189ms
  8   -8.300464592320       -7.24       -3.60    1.6   97.1ms
  9   -8.300464630079       -7.42       -3.80    1.8    106ms
 10   -8.300464638658       -8.07       -3.96    1.4    107ms
 11   -8.300464643311       -8.33       -4.21    5.9    207ms


In [4]:
self_consistent_field(aluminium_setup(2); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -16.67448506586                   -0.70    6.5         
  2   -16.67864289193       -2.38       -1.14    1.6    241ms
  3   -16.67919647411       -3.26       -1.86    3.5    251ms
  4   -16.67927597939       -4.10       -2.74    2.1    229ms
  5   -16.67928591669       -5.00       -3.17    6.1    351ms
  6   -16.67928620441       -6.54       -3.46    2.8    244ms
  7   -16.67928621804       -7.87       -3.93    2.0    195ms
  8   -16.67928622158       -8.45       -4.58    2.6    231ms


In [5]:
self_consistent_field(aluminium_setup(4); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -33.32532306603                   -0.56    7.1         
  2   -33.33269381138       -2.13       -1.00    1.0    701ms
  3   -33.33409256216       -2.85       -1.74    5.1    916ms
  4   -33.33568764638       -2.80       -2.25    2.4    763ms
  5   -33.33691354155       -2.91       -2.47   10.8    1.19s
  6   -33.33693503373       -4.67       -2.78    1.9    699ms
  7   -33.33694348995       -5.07       -3.59    2.5    797ms
  8   -33.33694363976       -6.82       -3.72    7.2    1.15s
  9   -33.33694375336       -6.94       -4.09    1.5    645ms


When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [6]:
self_consistent_field(aluminium_setup(1); tol=1e-4, mixing=SimpleMixing());

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.298272805202                   -0.85    5.0         
  2   -8.300244241668       -2.71       -1.59    1.0   59.4ms
  3   -8.300394606399       -3.82       -2.35    3.4   89.8ms
  4   -8.300325277076   +   -4.16       -2.18    2.5   87.3ms
  5   -8.300463992758       -3.86       -3.52    1.0   76.8ms
  6   -8.300464532214       -6.27       -3.71   11.6    201ms
  7   -8.300464631760       -7.00       -4.08    1.1   69.4ms


In [7]:
self_consistent_field(aluminium_setup(4); tol=1e-4, mixing=SimpleMixing());

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -33.32594399984                   -0.56    7.1         
  2   -33.32403140394   +   -2.72       -1.27    1.0    573ms
  3   -21.41183467853   +    1.08       -0.53    5.8    1.15s
  4   -33.29909996795        1.08       -1.72    6.8    1.27s
  5   -33.27188700947   +   -1.57       -1.47    2.9    868ms
  6   -33.10456094636   +   -0.78       -1.38    3.6    926ms
  7   -33.27182185425       -0.78       -1.63    4.5    948ms
  8   -33.33618064030       -1.19       -2.38    2.8    766ms
  9   -33.33606595178   +   -3.94       -2.54    3.9    951ms
 10   -33.33652474709       -3.34       -2.45    2.6    830ms
 11   -33.33686845864       -3.46       -3.04    2.0    728ms
 12   -33.33693437424       -4.18      

For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.